In [27]:
import pandas as pd
from IPython.display import HTML
import os
import ipywidgets as widgets
from IPython.display import display
import random as rnd
import collections
import itertools
import numpy as np
import time
import re1

pd.set_option('display.max_columns', 50)


class League:
    def __init__(self, title, teams, two_games=False):
        self.cur_round = 0
        self.title = title
        self.teams = teams
        self.two_games = two_games
        self.calendar = self.set_calendar()
        self.results = {}
        
    def set_calendar(self):
        teams = self.teams.index.to_list()
        teams_index = teams
        rnd.shuffle(teams_index)
        n = len(teams)
        tail = collections.deque(teams[1:n//2] + teams_index[n//2:][::-1])
        round_robin_home, round_robin_away = [],[]
        for _ in range(len(tail)):
            round_line = [teams_index[0], *tail]
            round_robin_home.append(pd.DataFrame({'abb_x':round_line[:n//2], 'abb_y':round_line[n//2:][::-1]})) 
            round_robin_away.append(pd.DataFrame({'abb_x':round_line[n//2:][::-1], 'abb_y':round_line[:n//2]}))   
            tail.rotate()   
        return round_robin_home + (round_robin_away if self.two_games else [])
    
    def round_results(self, round_num):
        res = self.calendar[round_num]\
            .merge(self.teams[['attack','defense']], left_on='abb_x', right_index=True, how='inner')\
            .merge(self.teams[['attack','defense']], left_on='abb_y', right_index=True, how='inner')
        res['GP_x'] = 1
        res['GP_y'] = 1
        res['GF_x'] = res.apply(lambda r: int(abs(rnd.normalvariate(r['attack_x'] - r['defense_y'], 1))), axis=1)
        res['GF_y'] = res.apply(lambda r: int(abs(rnd.normalvariate(r['attack_y'] - r['defense_x'], 1))), axis=1)
        res['GA_x'], res['GA_y'] = res['GF_y'], res['GF_x']
        res['+/-_x'] = res['GF_x'] - res['GA_x']
        res['+/-_y'] = res['GF_y'] - res['GA_y']
        res['W_x'] = np.where(res['GF_x'] > res['GA_x'], 1, 0)
        res['L_x'] = np.where(res['GF_x'] < res['GA_x'], 1, 0)
        res['T_x'] = np.where(res['GF_x'] == res['GA_x'], 1, 0)
        res['P_x'] = res['W_x']*3 + res['T_x']
        res['W_y'] = np.where(res['GF_y'] > res['GA_y'], 1, 0)
        res['L_y'] = np.where(res['GF_y'] < res['GA_y'], 1, 0)
        res['T_y'] = np.where(res['GF_y'] == res['GA_y'], 1, 0)
        res['P_y'] = res['W_y']*3 + res['T_y']
        res = res.drop(['attack_x','defense_x','attack_y','defense_y'], axis=1)
        self.results[round_num] = res
        
    def calc_table(self):
        for i,res in enumerate(self.results.values()):
            home = res[[x for x in res.columns if re.search('_x', x)]]
            away = res[[x for x in res.columns if re.search('_y', x)]]
            for x in [home, away]:
                x.columns = [c.split('_')[0] for c in x] 
            home_away = pd.concat([home,away]).set_index('abb') 
            if i == 0:
                table = home_away
            else:    
                table = table.add(home_away).combine_first(table)
        self.table = self.teams[['img']].join(table).sort_values(['P','+/-','W'], ascending=False)
        self.table.index.name = None
        
#     def elim_teams(self.teams):
#         league_a.teams.loc[league_a.table[-2:].index.to_list()]
               
    def table_html(self):
        return self.table.to_html(col_space='40px', justify='center', escape=False)
    
    def round_result_html(self, round_num):
        res = self.results[round_num]
        res = res.merge(self.teams[['img']], left_on='abb_x', right_index=True, how='inner')\
            .merge(self.teams[['img']], left_on='abb_y', right_index=True, how='inner', suffixes=('_x', '_y'))
        return res[['img_x','GF_x','img_y','GF_y']].to_html(index=False, col_space='40px', justify='center', escape=False)
    
teams = pd.read_csv('teams.csv', index_col='abb')    

In [175]:
for x in os.listdir('./images'):
    print(f'<img src="./images/{x}"/>,{x[:-4]}')

<img src="./images/Argentina.png"/>,Argentina
<img src="./images/Brazil.png"/>,Brazil
<img src="./images/England.png"/>,England
<img src="./images/France.png"/>,France
<img src="./images/Germany.png"/>,Germany
<img src="./images/Italy.png"/>,Italy
<img src="./images/Netherlands.png"/>,Netherlands
<img src="./images/Portugal.png"/>,Portugal
<img src="./images/Russia.png"/>,Russia
<img src="./images/Spain.png"/>,Spain


### Create cup

In [2]:
wTeams=widgets.SelectMultiple(options=teams['title'])
display(wTeams)

SelectMultiple(options=('Argentina', 'Brazil', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Portug…

In [43]:
teams_pool = teams[teams['title'].isin(wTeams.value)]
#teams_pool

,img,title,attack,defense
abb,,,,
ARG,<img src=./images/Argentina.png/>,Argentina,4.3,2.2
BRA,<img src=./images/Brazil.png/>,Brazil,4.5,2.4
ENG,<img src=./images/England.png/>,England,4.2,2.4
FRA,<img src=./images/France.png/>,France,4.6,2.6
GER,<img src=./images/Germany.png/>,Germany,4.6,2.5
ITA,<img src=./images/Italy.png/>,Italy,4.1,2.8
NTH,<img src=./images/Netherlands.png/>,Netherlands,4.3,2.6
POR,<img src=./images/Portugal.png/>,Portugal,4.1,2.7
RUS,<img src=./images/Russia.png/>,Russia,4.2,2.3


In [68]:
wTable = widgets.HTML(description='Lague A')
wResults = widgets.HTML(description='Round results')
display(wTable)
display(wResults)

league_a = League('League A', teams_pool, True)
for i,x in enumerate(league_a.calendar):
    league_a.round_results(i)
    league_a.calc_table()
    wTable.value = league_a.table_html()  
    wResults.value = league_a.round_result_html(i)
    time.sleep(2)      

HTML(value='', description='Lague A')

HTML(value='', description='Round results')

In [8]:
%timeit round_robin(cup.index.to_list(), True)

NameError: name 'round_robin' is not defined

In [17]:
league_a.teams.loc[league_a.table[-2:].index.to_list()]

,img,title,attack,defense
abb,,,,
ARG,<img src=./images/Argentina.png/>,Argentina,4.3,2.2
RUS,<img src=./images/Russia.png/>,Russia,4.2,2.3
